# Lecture 10: Sampling Rate & Quantization

**Goal:** Understand how sampling rate affects signal quality and learn to make engineering trade-offs.

**Time:** 15-20 minutes

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

## Part 1: Load and Visualize a Gesture

We'll use one "Backhand Tennis" gesture from a phone accelerometer.

In [ ]:
# Load one gesture
gesture_file = Path("./Backhand Tennis_Accelerometer_1523343205168.csv")
df = pd.read_csv(gesture_file)

# Parse data (Column 1=timestamp, Columns 3-5=x,y,z)
t_ms = df.iloc[:, 1].to_numpy()
X = df.iloc[:, 3:6].to_numpy()  # x, y, z accelerometer

# Convert time to seconds
t = (t_ms - t_ms[0]) / 1000.0

# Estimate sampling rate
dt = np.diff(t)
fs_est = 1.0 / np.median(dt)

print(f"Estimated sampling rate: {fs_est:.1f} Hz")
print(f"Number of samples: {len(t)}")
print(f"Duration: {t[-1]:.2f} seconds")

In [ ]:
# Plot original gesture
plt.figure(figsize=(12, 4))
plt.plot(t, X[:, 0], label='x', alpha=0.7)
plt.plot(t, X[:, 1], label='y', alpha=0.7)
plt.plot(t, X[:, 2], label='z', alpha=0.7)
plt.xlabel('Time (seconds)')
plt.ylabel('Acceleration')
plt.title(f'Original Gesture (sampled at {fs_est:.1f} Hz)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Quick Check

**Question:** Does 50 Hz seem reasonable for a phone sensor?
- ☐ Too slow (will miss details)
- ☐ About right
- ☐ Too fast (wasting power)

**Hint:** Human arm movements are relatively slow (~1-10 Hz)

---

## Part 2: Test Different Sampling Rates

Let's see what happens when we downsample to different rates.

In [ ]:
# Helper function: downsample to target rate
def downsample_uniform(t, X, fs_target):
    """Downsample by selecting samples at fixed intervals."""
    dt_target = 1.0 / fs_target
    t_new = np.arange(t[0], t[-1], dt_target)
    idx = np.searchsorted(t, t_new, side="left")
    idx = np.clip(idx, 0, len(t)-1)
    return t[idx], X[idx]

# Helper function: plot gesture
def plot_xyz(t, X, title):
    plt.figure(figsize=(12, 4))
    plt.plot(t, X[:, 0], 'o-', label='x', markersize=3, alpha=0.7)
    plt.plot(t, X[:, 1], 'o-', label='y', markersize=3, alpha=0.7)
    plt.plot(t, X[:, 2], 'o-', label='z', markersize=3, alpha=0.7)
    plt.xlabel('Time (seconds)')
    plt.ylabel('Acceleration')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# Test different sampling rates
test_rates = [100, 50, 25, 10]

print("Testing different sampling rates:")

for fs_target in test_rates:
    t_ds, X_ds = downsample_uniform(t, X, fs_target)
    
    # Show plot
    plot_xyz(t_ds, X_ds, f"Sampled at {fs_target} Hz")
    
    # Print stats
    n_samples = len(t_ds)
    reduction = len(t) / n_samples
    print(f"{fs_target:3d} Hz: {n_samples:3d} samples ({reduction:.1f}x reduction)")
    print()


---

## Part 3: Understanding Quantization

Quantization = rounding to discrete levels (like bit depth in ADC).

Let's see 8-bit (256 levels) vs 4-bit (16 levels).

In [ ]:
# Helper function: quantize to specified bit depth
def quantize_uniform(X, bits):
    """Quantize to 2^bits levels."""
    X = np.asarray(X, dtype=float)
    Xq = np.zeros_like(X)
    for k in range(X.shape[1]):
        x = X[:,k]
        lo = float(np.min(x))
        hi = float(np.max(x))
        levels = 2**bits
        x_clip = np.clip(x, lo, hi)
        q = np.round((x_clip - lo) / (hi - lo) * (levels - 1))
        Xq[:,k] = lo + q / (levels - 1) * (hi - lo)
    return Xq

In [ ]:
# Compare 8-bit vs 4-bit at 50 Hz
fs_target = 50
t_ds, X_ds = downsample_uniform(t, X, fs_target)

# Quantize to different bit depths
X_8bit = quantize_uniform(X_ds, 8)
X_4bit = quantize_uniform(X_ds, 4)

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# 8-bit
axes[0].plot(t_ds, X_8bit[:, 0], 'o-', markersize=3, label='x')
axes[0].set_title('8-bit Quantization (256 levels)')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Acceleration')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 4-bit
axes[1].plot(t_ds, X_4bit[:, 0], 'o-', markersize=3, label='x', color='orange')
axes[1].set_title('4-bit Quantization (16 levels)')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Acceleration')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Measure quantization error
error_8bit = np.mean(np.abs(X_ds[:, 0] - X_8bit[:, 0]))
error_4bit = np.mean(np.abs(X_ds[:, 0] - X_4bit[:, 0]))

print(f"Quantization error (average difference from original):")
print(f"  8-bit: {error_8bit:.4f}")
print(f"  4-bit: {error_4bit:.4f}")
print(f"  4-bit has {error_4bit/error_8bit:.1f}x more error")

---

## Part 4: The Worst Case

What happens when we combine low sampling rate AND low bit depth?

In [ ]:
# Worst case: 10 Hz + 4-bit
t_ds, X_ds = downsample_uniform(t, X, 10)
X_q = quantize_uniform(X_ds, 4)

plt.figure(figsize=(12, 4))
plt.plot(t_ds, X_q[:, 0], 'o-', label='x', markersize=6)
plt.plot(t_ds, X_q[:, 1], 'o-', label='y', markersize=6)
plt.plot(t_ds, X_q[:, 2], 'o-', label='z', markersize=6)
plt.xlabel('Time (seconds)')
plt.ylabel('Acceleration')
plt.title('Worst Case: 10 Hz + 4-bit Quantization')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Original: {len(t)} samples")
print(f"Degraded: {len(t_ds)} samples")
print(f"Data reduction: {len(t)/len(t_ds):.1f}× smaller")